<a href="https://colab.research.google.com/github/lilfetz22/Running_data/blob/Joining-Notes-columns/Running_Log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Garmin Data Wrangling

In [45]:
import pandas as pd
import re
import numpy as np
import math

In [46]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
def findall_n_join_fx(df, note_col, new_col, search_string):
    df.loc[:,new_col] = df.loc[:,note_col].str.findall(search_string)
    df.loc[:,new_col] = df.loc[:,new_col].str.join('')
    df.loc[:,note_col] = df.loc[:,note_col].str.replace(search_string, '')
    return df

In [ ]:
log_csv = pd.read_csv("/content/drive/My Drive/pre-garmin.csv",encoding='windows-1254')
log_csv.head(15)

,Running log 6/21/10,3/17/14,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Saturday 5/29/10,NaN,4 miles 27:19 it was a little short. 3.8 miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Monday 6/21/10,4 miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tuesday 6/22/10,4 miles,Swimming,10 laps of freestyle,5 laps of backstroke,5,laps of breaststroke,NaN,1 lap all out of freestyle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wednesday 6/23/10,4.5 miles,35 minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Thursday 6/25/10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Friday 6/26/10,5,Swimming,same as Tuesday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Saturday 6/27/10,6,41:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Monday 6/28/10,4,28:04:00,5 total,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Tuesday 6/29/10,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
log_csv.tail()

,Running log 6/21/10,3/17/14,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Notes
1979,TOTAL - 48.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,Monday - 12/21/15 - off,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981,Tuesday - I've failed. Not going to be able to...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,Thursday - 12/25/15 - 3 miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,Thursday - 12/31/15 - 30 minutes biking - 9.5 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
log_csv['Notes'] = log_csv['Unnamed: 2'].str.cat(log_csv[['Unnamed: 3', 'Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7',
                                                     'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11','Unnamed: 12',
                                                     'Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17']].astype(str), sep="_")
log_csv2 = log_csv[['Running log 6/21/10 ',' 3/17/14','Notes']]
log_csv2.head(10)

,Running log 6/21/10,3/17/14,Notes
0,Saturday 5/29/10,NaN,4 miles 27:19 it was a little short. 3.8 mile...
1,Monday 6/21/10,4 miles,NaN
2,Tuesday 6/22/10,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...
3,Wednesday 6/23/10,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...
4,Thursday 6/25/10,5,NaN
5,Friday 6/26/10,5,Swimming_ same as Tuesday_nan_nan_nan_nan_nan_...
6,Saturday 6/27/10,6,41:04_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan...
7,NaN,NaN,NaN
8,Monday 6/28/10,4,28:04:00_5 total_nan_nan_nan_nan_nan_nan_nan_n...
9,Tuesday 6/29/10,5,NaN


In [ ]:
# columns = ['Weekday', 'Date', 'Mileage', 'Minutes', 'Notes']
log_csv2.columns = ['Beginning', 'Mileage', 'Notes']

In [ ]:
m = '- 7 miles - warmup with maggie - 82.190 - 6:19:16 - 13.0 avg'
t = '- 8 miles - thanksgiving 18.229 mile'
e = '35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan'
re.sub('_nan', '', e)

'35 minutes'

In [ ]:
findall_n_join_fx(log_csv2, 'Beginning', 'Weekday', '\w+day')
log_csv2 = findall_n_join_fx(log_csv2, 'Beginning', 'Date', '\d+/\d+/\d\d')
log_csv2.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,Beginning,Mileage,Notes,Weekday,Date
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10
1,,4 miles,NaN,Monday,6/21/10
2,,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10
3,,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10
4,,5,NaN,Thursday,6/25/10


In [ ]:
log_csv2.sample(20)

,Beginning,Mileage,Notes,Weekday,Date
1647,- 13 miles,NaN,NaN,Thursday,
1894,TOTAL -,NaN,NaN,,
492,,3k course,1 miles race pace _ 5:15ish slow _ 4x400 _ 4x...,Wednesday,
188,,6 miles,NaN,Friday,
1173,,8 miles,7:04 pace _nan_nan_nan_nan_nan_nan_nan_nan_na...,Friday,
1860,TOTAL -,NaN,NaN,,
1372,- - 7 miles - 20.771 - 1:35:34,NaN,NaN,Monday,6/30/14
296,,6 miles,same as yesterday_nan_nan_nan_nan_nan_nan_nan...,Wednesday,
888,,3 mile tempo,___ pace _ 8 miles total _nan_nan_nan_nan_nan...,Friday,
1134,,3.5 miles went a little far,NaN,Tuesday,


In [ ]:
f_col = list(log_csv2.Beginning)
for row in f_col:
    if (row != '') or (row != ' ') or row != 'nan':
        print(row)

In [ ]:
log_csv3 = log_csv2[log_csv2['Beginning'].str.startswith('TOTAL') == False]

In [ ]:
log_csv3.loc[:,'Mileage_init'] = log_csv3.loc[:,'Beginning'].str.findall('\d+\.*\d*')
log_csv3.loc[:, 'length'] = log_csv3['Mileage_init'].str.len()
log_csv3['counter'] = range(len(log_csv3))
log_csv3['Workout_mins'] = np.nan
log_csv3['Swimming_meters'] = np.nan
log_csv3['Bike_Mileage'] = np.nan
log_csv3 = log_csv3.rename(columns = {'Beginning': 'Notes2'})
check_data = log_csv3[log_csv3.length >1]
check_data2 = check_data[['Notes2','Mileage','Mileage_init','counter']]
len(check_data2)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

157

In [ ]:
check_data2 = check_data2.reset_index()

In [ ]:
# log_csv3.to_csv("/content/drive/My Drive/mileage_manip.csv")

In [197]:
new_log = pd.read_csv("/content/drive/My Drive/mileage_manip.csv",index_col=0)
new_log.head(10)

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_mins,Swimming_meters,Bike_Mileage
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,[],0,0,NaN,NaN,NaN
1,,4 miles,NaN,Monday,6/21/10,[],0,1,NaN,NaN,NaN
2,,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,[],0,2,NaN,NaN,NaN
3,,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,[],0,3,NaN,NaN,NaN
4,,5,NaN,Thursday,6/25/10,[],0,4,NaN,NaN,NaN
5,,5,Swimming_ same as Tuesday_nan_nan_nan_nan_nan_...,Friday,6/26/10,[],0,5,NaN,NaN,NaN
6,,6,41:04_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Saturday,6/27/10,[],0,6,NaN,NaN,NaN
8,,4,28:04:00_5 total_nan_nan_nan_nan_nan_nan_nan_n...,Monday,6/28/10,[],0,7,NaN,NaN,NaN
9,,5,NaN,Tuesday,6/29/10,[],0,8,NaN,NaN,NaN
10,,5.5,Swimming_normal_nan_nan_nan_nan_nan_nan_nan_na...,Wednesday,6/30/10,[],0,9,NaN,NaN,NaN


In [198]:
to_be_nan = list(new_log.counter[new_log.length > 1])
for row_numb in to_be_nan:
    new_log.iloc[row_numb,5] = np.nan

In [199]:
new_log.loc[:,'Mileage_init'] = new_log.loc[:,'Mileage_init'].str.replace('\[\'', '')
new_log.loc[:,'Mileage_init'] = new_log.loc[:,'Mileage_init'].str.replace('\'\]', '')
new_log.loc[:,'Mileage_init'] = new_log.loc[:,'Mileage_init'].str.replace('\[\]', '')

In [200]:
new_log.Mileage_init = pd.to_numeric(new_log.Mileage_init, errors='coerce')
new_log.head()

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_mins,Swimming_meters,Bike_Mileage
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,NaN,0,0,NaN,NaN,NaN
1,,4 miles,NaN,Monday,6/21/10,NaN,0,1,NaN,NaN,NaN
2,,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,NaN,NaN,NaN
3,,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,NaN,0,3,NaN,NaN,NaN
4,,5,NaN,Thursday,6/25/10,NaN,0,4,NaN,NaN,NaN


In [201]:
new_log.insert(8, 'Workout_type',None)

In [202]:
new_log.head()

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,NaN,0,0,None,NaN,NaN,NaN
1,,4 miles,NaN,Monday,6/21/10,NaN,0,1,None,NaN,NaN,NaN
2,,4 miles,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,None,NaN,NaN,NaN
3,,4.5 miles,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,NaN,0,3,None,NaN,NaN,NaN
4,,5,NaN,Thursday,6/25/10,NaN,0,4,None,NaN,NaN,NaN


In [203]:
# dropping two lines that just had "WORKOUTS TOTAL"
new_log = new_log.drop([1727,1747])
new_log['counter'] = range(len(new_log))

In [204]:
def workout_finder_fx(df, col, str_search, minutes, workout_type, *exclude):
    w_o = list(df.loc[:, col].str.contains(str_search))
    for idx, n in enumerate(w_o):
        if n == None:
            continue
        elif n == True:
            enough_to_exclude = 0
            for e in exclude:
                o2 = re.search(e, df[col].iloc[idx])
                if o2:
                    enough_to_exclude += 1
            if enough_to_exclude < 1:
                df['Workout_type'].iloc[idx] = workout_type
                df['Workout_mins'].iloc[idx] = minutes
    return df



In [205]:
ab_workout_incl_log = workout_finder_fx(new_log, 'Notes2', '[Aa]b [Ww]orkout', 20, 'abs')
gym_workout_incl_log = workout_finder_fx(ab_workout_incl_log, 'Notes2', '[Ww]orkout',45, 'gym', '[Ss]wim', '[Aa]b')
print(gym_workout_incl_log.iloc[1084])
gym_workout_incl_log.iloc[1267]

Notes2              - 4 miles - ab workout - ran with Keith
Mileage                                                 NaN
Notes                                                   NaN
Weekday                                             Tuesday
Date                                                    NaN
Mileage_init                                              4
length                                                    1
counter                                                1084
Workout_type                                            abs
Workout_mins                                             20
Swimming_meters                                         NaN
Bike_Mileage                                            NaN
Name: 1264, dtype: object


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Notes2              - 14 miles - workout between classes
Mileage                                              NaN
Notes                                                NaN
Weekday                                          Tuesday
Date                                                 NaN
Mileage_init                                          14
length                                                 1
counter                                             1267
Workout_type                                         gym
Workout_mins                                          45
Swimming_meters                                      NaN
Bike_Mileage                                         NaN
Name: 1472, dtype: object

Mileage Column

In [206]:
gym_workout_incl_log.iloc[314]

Notes2                                                              
Mileage                                                     8/29/11 
Notes               warmup _ 20 mins inside _ 2.5ish _ 2 with gir...
Weekday                                                       Monday
Date                                                             NaN
Mileage_init                                                     NaN
length                                                             0
counter                                                          314
Workout_type                                                    None
Workout_mins                                                     NaN
Swimming_meters                                                  NaN
Bike_Mileage                                                     NaN
Name: 371, dtype: object

In [207]:
# might need to try out the "try" in a function to see if it can be converted to 
# an integer if not, then put the note into a separate column and delete it from
# the mileage column
def remove_mileage_notes_fx(df, col):
    notes3_col = [''] * len(df[col])
    list_col = list(df.loc[:, col])
    for idx, a in enumerate(list_col):
        try:
            float_a = float(a)
            df[col].iloc[idx] = float_a
        except:
            notes3_col[idx] = a
            df[col].iloc[idx] = np.nan
    df['Notes3'] = notes3_col
    return df
sep_notes_from_mileage_col = remove_mileage_notes_fx(gym_workout_incl_log, 'Mileage')
sep_notes_from_mileage_col.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage,Notes3
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,5/29/10,NaN,0,0,None,NaN,NaN,NaN,
1,,NaN,NaN,Monday,6/21/10,NaN,0,1,None,NaN,NaN,NaN,4 miles
2,,NaN,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,6/22/10,NaN,0,2,None,NaN,NaN,NaN,4 miles
3,,NaN,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,6/23/10,NaN,0,3,None,NaN,NaN,NaN,4.5 miles
4,,5,NaN,Thursday,6/25/10,NaN,0,4,None,NaN,NaN,NaN,
5,,5,Swimming_ same as Tuesday_nan_nan_nan_nan_nan_...,Friday,6/26/10,NaN,0,5,None,NaN,NaN,NaN,
6,,6,41:04_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Saturday,6/27/10,NaN,0,6,None,NaN,NaN,NaN,
8,,4,28:04:00_5 total_nan_nan_nan_nan_nan_nan_nan_n...,Monday,6/28/10,NaN,0,7,None,NaN,NaN,NaN,
9,,5,NaN,Tuesday,6/29/10,NaN,0,8,None,NaN,NaN,NaN,
10,,5.5,Swimming_normal_nan_nan_nan_nan_nan_nan_nan_na...,Wednesday,6/30/10,NaN,0,9,None,NaN,NaN,NaN,


took all of the text that was in the mileage column and moved it to a separate column, and left any number that was purely a number and converted the column to be integers. 

In [208]:
# now I need to go through the notes3 column and pull out any that have just mileage, 
#then I will need to go through and search those that have dates and other mess 
#in there, then I will need to apply the same functions to the Notes column

In [209]:
findall_n_join_fx(sep_notes_from_mileage_col, 'Notes3', 'Date', '\d+/\d+/\d\d')

,Notes2,Mileage,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage,Notes3
0,,NaN,4 miles 27:19 it was a little short. 3.8 mile...,Saturday,,NaN,0,0,None,NaN,NaN,NaN,
1,,NaN,NaN,Monday,,NaN,0,1,None,NaN,NaN,NaN,4 miles
2,,NaN,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,,NaN,0,2,None,NaN,NaN,NaN,4 miles
3,,NaN,35 minutes_nan_nan_nan_nan_nan_nan_nan_nan_nan...,Wednesday,,NaN,0,3,None,NaN,NaN,NaN,4.5 miles
4,,5,NaN,Thursday,,NaN,0,4,None,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,- 10 miles,NaN,NaN,Sunday,,10.0,1,1716,None,NaN,NaN,NaN,
1980,- - off,NaN,NaN,Monday,,NaN,0,1717,None,NaN,NaN,NaN,
1981,- I've failed. Not going to be able to run fo...,NaN,NaN,Tuesday,,NaN,0,1718,None,NaN,NaN,NaN,
1982,- - 3 miles,NaN,NaN,Thursday,,3.0,1,1719,None,NaN,NaN,NaN,


In [210]:
sep_notes_from_mileage_col.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1983
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Notes2           1663 non-null   object 
 1   Mileage          192 non-null    object 
 2   Notes            741 non-null    object 
 3   Weekday          1618 non-null   object 
 4   Date             1721 non-null   object 
 5   Mileage_init     397 non-null    float64
 6   length           1721 non-null   int64  
 7   counter          1721 non-null   int64  
 8   Workout_type     68 non-null     object 
 9   Workout_mins     70 non-null     float64
 10  Swimming_meters  18 non-null     float64
 11  Bike_Mileage     112 non-null    float64
 12  Notes3           1721 non-null   object 
dtypes: float64(4), int64(2), object(7)
memory usage: 188.2+ KB


In [211]:
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.replace('_nan', '')

In [212]:
sep_notes_from_mileage_col.iloc[2,2].lower()


'swimming_ 10 laps of freestyle_5 laps of backstroke_5_laps of breaststroke_1 lap all out of freestyle'

In [213]:
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.lower()
sep_notes_from_mileage_col.loc[:, 'Notes2'] = sep_notes_from_mileage_col.loc[:, 'Notes2'].str.lower()
sep_notes_from_mileage_col.loc[:, 'Notes3'] = sep_notes_from_mileage_col.loc[:, 'Notes3'].str.lower()

In [214]:
sep_notes_from_mileage_col['Mileage_init2'] = sep_notes_from_mileage_col.Notes3.str.findall('\d+\.?\d* miles?')
sep_notes_from_mileage_col.loc[:, 'Notes3'] = sep_notes_from_mileage_col.loc[:, 'Notes3'].str.replace('\d+\.?\d* miles?', '')
sep_notes_from_mileage_col['Mileage_init3'] = sep_notes_from_mileage_col.Notes.str.findall('\d+\.?\d* miles?')
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.replace('\d+\.?\d* miles?', '')
sep_notes_from_mileage_col['Mileage_init4'] = sep_notes_from_mileage_col.Notes3.str.findall('\d+\.?\d* total')
sep_notes_from_mileage_col.loc[:, 'Notes3'] = sep_notes_from_mileage_col.loc[:, 'Notes3'].str.replace('\d+\.?\d* total', '')
sep_notes_from_mileage_col['Mileage_init5'] = sep_notes_from_mileage_col.Notes.str.findall('\d+\.?\d* total')
sep_notes_from_mileage_col.loc[:, 'Notes'] = sep_notes_from_mileage_col.loc[:, 'Notes'].str.replace('\d+\.?\d* total', '')

In [225]:
gym_workout_incl_log2 = workout_finder_fx(sep_notes_from_mileage_col, 'Notes', 'workout', 45, 'gym', 'swim', 'ab','pool')
gym_workout_incl_log2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1983
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Notes2           1663 non-null   object 
 1   Mileage          192 non-null    object 
 2   Notes            741 non-null    object 
 3   Weekday          1618 non-null   object 
 4   Date             1721 non-null   object 
 5   Mileage_init     397 non-null    float64
 6   length           1721 non-null   int64  
 7   counter          1721 non-null   int64  
 8   Workout_type     73 non-null     object 
 9   Workout_mins     75 non-null     float64
 10  Swimming_meters  18 non-null     float64
 11  Bike_Mileage     112 non-null    float64
 12  Notes3           1721 non-null   object 
 13  Mileage_init2    1721 non-null   object 
 14  Mileage_init3    741 non-null    object 
 15  Mileage_init4    1721 non-null   object 
 16  Mileage_init5    741 non-null    object 
dtypes: float64(4),

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [239]:
def swim_workout_fx(df, col, str_search): 
    for s in str_search:
        w_o2 = list(df.loc[:, col].str.contains(s))
        for idx, w in enumerate(w_o2):
            if w == None:
                continue
            elif w == True:
                df['Swimming_meters'].iloc[idx] = 1600

In [240]:
swim_workout_fx(gym_workout_incl_log2, 'Notes', ['swim','pool'])
swim_workout_fx(gym_workout_incl_log2, 'Notes2', ['swim','pool'])
swim_workout_fx(gym_workout_incl_log2, 'Notes3', ['swim','pool'])
gym_workout_incl_log2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1721 entries, 0 to 1983
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Notes2           1663 non-null   object 
 1   Mileage          192 non-null    object 
 2   Notes            741 non-null    object 
 3   Weekday          1618 non-null   object 
 4   Date             1721 non-null   object 
 5   Mileage_init     397 non-null    float64
 6   length           1721 non-null   int64  
 7   counter          1721 non-null   int64  
 8   Workout_type     73 non-null     object 
 9   Workout_mins     75 non-null     float64
 10  Swimming_meters  38 non-null     float64
 11  Bike_Mileage     112 non-null    float64
 12  Notes3           1721 non-null   object 
 13  Mileage_init2    1721 non-null   object 
 14  Mileage_init3    741 non-null    object 
 15  Mileage_init4    1721 non-null   object 
 16  Mileage_init5    741 non-null    object 
dtypes: float64(4),

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [215]:
def mileage_finder_fx(df, init, total_or_miles):
    multiple_miles = []
    too_big = []
    already_in_col = []
    for idx, row in enumerate(df.loc[:, init]):
        try:
            if math.isnan(row):
                continue
        except:
            if len(row) > 1:
                multiple_miles.append(idx)
            elif len(row) == 1:
                mile_str = row[0].replace(total_or_miles, '')
                try:
                    mile_float = float(mile_str)
                    if math.isnan(df['Mileage'].iloc[idx]):
                        if mile_float > 27.2:
                            too_big.append(idx)
                        else:
                            df['Mileage'].iloc[idx] = mile_float
                    else:
                        already_in_col.append(idx)
                except:
                    continue
    return multiple_miles, too_big, already_in_col



In [195]:
mm, tb, aic = mileage_finder_fx(gym_workout_incl_log2,'Mileage_init2','miles')
print(len(mm),len(tb),len(aic))
mm2, tb2, aic2 = mileage_finder_fx(gym_workout_incl_log2, 'Mileage_init3', 'miles')
print(len(mm2),len(aic2),len(tb2))
mm3, tb3, aic3 = mileage_finder_fx(gym_workout_incl_log2,'Mileage_init4','total')
print(len(mm3),len(tb3),len(aic3))
mm4, tb4, aic4 = mileage_finder_fx(gym_workout_incl_log2, 'Mileage_init5', 'total')
print(len(mm4),len(aic4),len(tb4))

5 4 0
67 15 0
0 0 1
1 111 0


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


For each of the following I will be manually putting in the data to fix the issue since they are outliers in being able to conform nicely to a computer function. 

In [ ]:
def manual_input_desc(df, list_name):
    for ind in list_name:
        print(df[['Mileage','counter']].iloc[ind])
        print(df['Notes3'].iloc[ind], df['Notes'].iloc[ind])
manual_input_desc(gym_workout_incl_log2, mm)

Mileage    NaN
counter     29
Name: 34, dtype: object
2 miles / 5 miles hike nan
Mileage    NaN
counter     45
Name: 53, dtype: object
3 miles in the morning 5 miles in the evening  nan
Mileage    NaN
counter    429
Name: 505, dtype: object
 3 miles and 24 miles bike and 3 around greystone   6 total_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan
Mileage    NaN
counter    656
Name: 769, dtype: object
 7.4 miles in am and 7.6 miles in pm   15 total_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan
Mileage     NaN
counter    1069
Name: 1247, dtype: object
 FIRE TOWER!!!! Ran 6 miles with Maggie than ran 3 miles to trail then 3.5 to cone manor then ran 2.5 up to the fire tower   so beautiful up there _ but misjudged the mileage so on way down tried to take a shortcut through trout lake _ it ended up being a lot longer than 18 miles cuz I hit 18 around trout lake and then at least another 3 miles _ 21 miles _nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan


In [ ]:
gym_workout_incl_log2.iloc[29, 1] = 2
gym_workout_incl_log2.iloc[45, 1] = 8
gym_workout_incl_log2.iloc[429, 1] = 6
gym_workout_incl_log2.iloc[656, 1] = 15
gym_workout_incl_log2.iloc[1069, 1] = 21

In [ ]:
manual_input_desc(gym_workout_incl_log2, tb)

Mileage    NaN
counter    865
Name: 1013, dtype: object
 30 miles  nan
Mileage    NaN
counter    872
Name: 1021, dtype: object
 30 miles  nan
Mileage    NaN
counter    877
Name: 1026, dtype: object
 8 .75 miles   sawmill to creedmoor to Strickland to falls to sawmill _ 9.75 total_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan_nan
Mileage    NaN
counter    879
Name: 1029, dtype: object
 30 miles  nan


In [ ]:
gym_workout_incl_log2.iloc[865, -3] = 30
gym_workout_incl_log2.iloc[872, -3] = 30
gym_workout_incl_log2.iloc[877,1] = 9.75
gym_workout_incl_log2.iloc[879, -3] = 30

In [ ]:
gym_workout_incl_log2['numbers'] = gym_workout_incl_log2.Notes3.str.findall('\d+\.*\d*')

In [ ]:
greater_2 = []
two = []
one = []
for idx, num in enumerate(gym_workout_incl_log2.numbers):
    if len(num) > 2:
        greater_2.append(idx)
    elif len(num) == 2:
        two.append(idx)
    elif len(num) == 1:
        one.append(idx)
print(len(greater_2), len(two), len(one))



25 33 114


In [ ]:
time_added = gym_workout_incl_log2.copy()
time_added.insert(2, 'Time', None)

More manual inputting of data since these numbers could litterally mean anything since they didn't have the "miles" designation they weren't originally caught in the first analysis, but they also have a multitude of possibilities

In [ ]:
manual_input_desc(time_added,two)

Mileage    NaN
counter     30
Name: 35, dtype: object
 race: 11:29 5 miles total
Mileage     6
counter    34
Name: 40, dtype: object
 40:47 7 total
Mileage    NaN
counter     35
Name: 41, dtype: object
5x1200 > 3:59,3:59,4:00,4:05,4:02_5 miles total
Mileage     4
counter    39
Name: 46, dtype: object
 + 3k course = 7miles total nan
Mileage     6
counter    41
Name: 48, dtype: object
 42:59 7 total
Mileage     7
counter    52
Name: 61, dtype: object
 50:50 8 total
Mileage    NaN
counter     70
Name: 89, dtype: object
 54 mins 9 total nan
Mileage    NaN
counter     76
Name: 96, dtype: object
 5k+3k  33 mins _ 6 total
Mileage     6
counter    77
Name: 97, dtype: object
 leesville meet 19:11 paced first two miles  total  nan
Mileage    NaN
counter     89
Name: 111, dtype: object
3k and 5k   6 total
Mileage    NaN
counter    103
Name: 127, dtype: object
 4x800   2:52 _ 4x300 _ 48,49,47,48 _ 5 miles 
Mileage    NaN
counter    149
Name: 181, dtype: object
 8.5 64 mins nan
Mileage    8.4
count

In [ ]:
manual_changes_mileage1 = [[27,9.25],[47,6],[51,6],[53,6.5],[56,15],[57,6],[59,5],[61,4],[69,5],[78,9.5],[80,4.2],
                  [81,7],[83,6],[106,5],[153,10],[190,6],[256,10],[266,10.5],[318,4],[504,4],[552,3],
                  [672,14.1],[903,12.67],[917,8],[30,5],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
manual_changes_time2 = [[51,'29:30'],[56,'1:43:21'],[78,'60:47'],[81,'50:00'],[153,'1:03:00'],[256,'1:08:58'],
                       [266,'1:11:23'],[672,'1:11:23']]
manual_changes_date6 = [[552,'6/17/12']]
time_added.iloc[552,5] = '6/17/12'


In [ ]:
time_added.head()

,Notes2,Mileage,Time,Notes,Weekday,Date,Mileage_init,length,counter,Workout_type,Workout_mins,Swimming_meters,Bike_Mileage,Notes3,Mileage_init2,numbers
0,,NaN,None,4 miles 27:19 it was a little short. 3.8 miles,Saturday,,NaN,0,0,None,NaN,NaN,NaN,,[],[]
1,,4,None,NaN,Monday,,NaN,0,1,None,NaN,NaN,NaN,,[4 miles],[]
2,,4,None,Swimming_ 10 laps of freestyle_5 laps of backs...,Tuesday,,NaN,0,2,None,NaN,NaN,NaN,,[4 miles],[]
3,,4.5,None,35 minutes,Wednesday,,NaN,0,3,None,NaN,NaN,NaN,,[4.5 miles],[]
4,,5,None,NaN,Thursday,,NaN,0,4,None,NaN,NaN,NaN,,[],[]


# Garmin Data Wrangling

# Future Garmin Data Web Scraping

In [ ]:
!pip install requests

In [ ]:
!pip install beautifulsoup4

In [ ]:
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

In [ ]:
URL = "https://connect.garmin.com/modern/activities"

requests.get(URL)

<Response [200]>

In [ ]:
requests.get(URL, {}).text

'<!DOCTYPE html><html><head><meta charSet="utf-8"/><meta http-equiv="x-ua-compatible" content="ie=edge"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/><style data-href="/styles.3bfbd3dd9436b814ee1a.css">body,html{height:auto;font:400 14px/22px Open Sans,Helvetica Neue,Helvetica,Arial,sans-serif;color:#222;background-color:#efefef;margin:0}a,a:active,a:hover,a:visited{text-decoration:none;color:inherit}#truste-consent-track{display:block}#truste-consent-content{display:flex;align-items:center;justify-content:center;flex-wrap:wrap;width:100%;box-sizing:border-box}#truste-consent-text{margin:0 1rem 0 0}#truste-consent-buttons{display:flex;flex-direction:row;margin-top:1px!important;margin-left:10px!important}#truste-consent-button,#truste-show-consent{font-family:Open Sans,HelveticaNeue,HelveticaNeueu,Arial,sans-serif;font-size:14px;color:#fff;background-color:#4a4a4a;cursor:pointer;padding:5px 20px;margin:12px 0;border-radius:4px;border:none}#trus

In [ ]:
web_page = bs4.BeautifulSoup(requests.get(URL, {}).text, "lxml")

In [ ]:
web_page

<!DOCTYPE html>
<html><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><style data-href="/styles.3bfbd3dd9436b814ee1a.css">body,html{height:auto;font:400 14px/22px Open Sans,Helvetica Neue,Helvetica,Arial,sans-serif;color:#222;background-color:#efefef;margin:0}a,a:active,a:hover,a:visited{text-decoration:none;color:inherit}#truste-consent-track{display:block}#truste-consent-content{display:flex;align-items:center;justify-content:center;flex-wrap:wrap;width:100%;box-sizing:border-box}#truste-consent-text{margin:0 1rem 0 0}#truste-consent-buttons{display:flex;flex-direction:row;margin-top:1px!important;margin-left:10px!important}#truste-consent-button,#truste-show-consent{font-family:Open Sans,HelveticaNeue,HelveticaNeueu,Arial,sans-serif;font-size:14px;color:#fff;background-color:#4a4a4a;cursor:pointer;padding:5px 20px;margin:12px 0;border-radius:4px;border:none}#trus

In [ ]:
web_page.head.title

<title data-react-helmet="true">Garmin Connect | </title>

In [ ]:
web_page.body

<body><noscript id="gatsby-noscript">This app works best with JavaScript enabled.</noscript><div id="___gatsby"><div role="group" style="outline:none" tabindex="-1"><div><header class="c017"><h1 class="c0116 c0118"><a class="c018" href="/" title="Connect"></a></h1><div class="c0114"></div></header><main class="c014 c016"><div class="c013"><div class="c011"><div class="c012" id="gauth-widget"><noscript>To sign in to your Garmin account, please enable JavaScript in your web browser.</noscript></div></div><div id="loginGauth"></div></div></main><footer class="c0123"><div></div><div class="c0132"><span class="c0129 c0133"><a href="https://www.instagram.com/garminfitness/" title="Instagram"></a></span><span class="c0130 c0134"><a href="https://www.facebook.com/garminfit" title="Facebook"></a></span><span class="c0131 c0135"><a href="https://twitter.com/garminfitness" title="Twitter"></a></span></div><div id="consent"></div></footer></div></div></div><script id="gatsby-script-loader">/*<![CD

In [ ]:
sub_webpage = web_page.find_all(name="li") #, attrs={'class': 'list-item animated row-fluid'}

In [ ]:
sub_webpage

[]

I would like to make searchable notes that I could go through with links to exercises for injuries and such. It would be really useful to be able to have the types of injuries that I have gone through before and be able to just immediately have the stretches and such that worked. Then I can the next time find out how that went and then adjust based upon the previous injury timeline and see what works best for recovery for me. 